In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline                               


In [33]:
path = "./FIDs-features/"

def process(df, rType):
    df['FID'] = df['p1'].str[:5]
    df['p1'] = path+df['p1']
    df['p2'] = path+df['p2']
    df['p1'] = df['p1'].str.replace('.jpg', '.pkl')
    df['p2'] = df['p2'].str.replace('.jpg', '.pkl')
    
    sample1 = df.sample(n=df.shape[0], replace=True)[['p1','FID']].reset_index().drop('index',axis=1)
    sample2 = df.sample(n=df.shape[0], replace=True)[['p2','FID']].reset_index().drop('index',axis=1)
    sample2.columns = ['p2','FID2']
    unrelated = pd.concat([sample1,sample2],axis=1)
    unrelated = unrelated[unrelated['FID'] != unrelated['FID2']][['p1','p2']]
    
    ###############################################################
    # Related file
    p1 = df['p1'].values.tolist()
    p2 = df['p2'].values.tolist()
    F = pd.read_pickle(p1[0]).reshape(1,512)
    C = pd.read_pickle(p2[0]).reshape(1,512)
    related_fs_set = np.append(F,C,axis=1)

    for r1, r2 in zip(p1[1:],p2[1:]):
        F = pd.read_pickle(r1).reshape(1,512)
        C = pd.read_pickle(r2).reshape(1,512)
        temp = np.append(F,C,axis=1)
        related_fs_set = np.append(related_fs_set, temp, axis=0)
        
    pd.DataFrame(related_fs_set).to_pickle("Related_{}_1024.pkl".format(rType))
        
    ###############################################################
    # UnRelated file
    p1 = unrelated['p1'].values.tolist()
    p2 = unrelated['p2'].values.tolist()
    F = pd.read_pickle(p1[0]).reshape(1,512)
    C = pd.read_pickle(p2[0]).reshape(1,512)
    unrelated_fs_set = np.append(F,C,axis=1)

    for r1, r2 in zip(p1[1:],p2[1:]):
        F = pd.read_pickle(r1).reshape(1,512)
        C = pd.read_pickle(r2).reshape(1,512)
        temp = np.append(F,C,axis=1)
        unrelated_fs_set = np.append(unrelated_fs_set, temp, axis=0)
        
    pd.DataFrame(unrelated_fs_set).to_pickle("Unrelated_{}_1024.pkl".format(rType))
  
# Example 
process(pd.read_pickle("./lists/pairs/pickles/Direct/fs-faces.pkl"), "fs")
process(pd.read_pickle("./lists/pairs/pickles/Direct/fd-faces.pkl"), "fd")
process(pd.read_pickle("./lists/pairs/pickles/Direct/ms-faces.pkl"), "ms")
process(pd.read_pickle("./lists/pairs/pickles/Direct/md-faces.pkl"), "md")

In [5]:
p1 = pd.read_pickle("./Related_fs_1024.pkl")
p1['class'] = 1
p2 = pd.read_pickle("./Unrelated_fs_1024.pkl")
p2['class'] = 0
df_fs = pd.concat([p1, p2])
df_fs['label'] = 'fs'
#################################################################

p1 = pd.read_pickle("./Related_fd.pkl")
p1['class'] = 1
p2 = pd.read_pickle("./Unrelated_fd.pkl")
p2['class'] = 0
df_fd = pd.concat([p1, p2])
df_fd['label'] = 'fd'
#################################################################

p1 = pd.read_pickle("./Related_ms.pkl")
p1['class'] = 1
p2 = pd.read_pickle("./Unrelated_ms.pkl")
p2['class'] = 0
df_ms = pd.concat([p1, p2])
df_ms['label'] = 'ms'

#################################################################
p1 = pd.read_pickle("./Related_md.pkl")
p1['class'] = 1
p2 = pd.read_pickle("./Unrelated_md.pkl")
p2['class'] = 0
df_md = pd.concat([p1, p2])
df_md['label'] = 'md'


df = pd.concat([df_fs, df_fd, df_ms, df_md])

In [6]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,1016,1017,1018,1019,1020,1021,1022,1023,class,label
0,0.054371,-0.014020,-0.040592,0.001152,-0.005127,-0.032224,0.036430,-0.002104,-0.015939,-0.075741,...,0.024401,0.028892,-0.025882,-0.001901,-0.046780,0.042080,0.002879,0.019761,1,fs
1,0.042371,0.035194,0.034465,-0.014003,0.058805,0.022217,-0.033169,-0.053590,-0.010134,0.074855,...,0.064253,0.025173,-0.060588,-0.049711,0.077463,0.082833,-0.025950,-0.009140,1,fs
2,0.020210,-0.008038,-0.020024,0.001893,-0.061824,-0.019616,0.067781,0.007606,-0.004513,-0.060691,...,0.015091,0.022699,-0.129402,0.000146,0.004989,0.022917,-0.032020,0.062286,0,fs
3,0.010921,0.045188,-0.015183,0.029990,-0.030451,-0.062793,-0.023287,0.041561,0.071418,-0.038416,...,0.037846,-0.014511,0.134132,0.078374,0.013405,0.001500,0.043978,-0.050253,0,fs
4,0.090976,-0.022020,0.053715,-0.052504,-0.007729,0.001302,0.014140,-0.024895,0.017223,0.031572,...,0.050355,-0.062818,-0.094613,0.015777,0.023049,-0.018808,0.099998,-0.003690,0,fs


In [7]:
df.shape

(91159, 1026)

## Model Development

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [10]:
X_train = df.drop(['label','class'],axis=1)[0:50000].values.astype(np.float64)
X_valid = df.drop(['label','class'],axis=1)[50000:70000].values.astype(np.float64)
X_test = df.drop(['label','class'],axis=1)[70000:].values.astype(np.float64)

X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

y_train = df['class'][0:50000].values.astype(np.int64)
y_valid = df['class'][50000:70000].values.astype(np.int64)
y_test = df['class'][70000:].values.astype(np.int64)

train_dataset = TensorDataset(torch.from_numpy(X_train).type(torch.FloatTensor), torch.from_numpy(y_train))
valid_dataset = TensorDataset(torch.from_numpy(X_valid).type(torch.FloatTensor), torch.from_numpy(y_valid))
test_dataset = TensorDataset(torch.from_numpy(X_test).type(torch.FloatTensor), torch.from_numpy(y_test))

loaders = {}
loaders['train'] = DataLoader(train_dataset, batch_size=200, shuffle=True)
loaders['valid'] = DataLoader(valid_dataset, batch_size=200)
loaders['test'] = DataLoader(test_dataset, batch_size=200)

In [28]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 64)
        self.output = nn.Linear(64, 2)
        
        self.dropout = nn.Dropout(p=0.5)
        self.logSoftMax = nn.LogSoftmax(dim=1)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.dropout(self.relu(self.fc3(x)))
        x = self.logSoftMax(self.output(x))
        return x

model = Net()
use_cuda = torch.cuda.is_available()
if use_cuda:
    model.cuda()

In [29]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [30]:
"""returns trained model"""
# initialize tracker for minimum validation loss
valid_loss_min = np.Inf 

for epoch in range(200):
    # initialize variables to monitor training and validation loss
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        optimizer.zero_grad()
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        ## find the loss and update the model parameters accordingly
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        ## record the average training loss, using something like
        train_loss += ((1 / (batch_idx + 1)) * (loss.data - train_loss))

    ######################    
    # validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['valid']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        ## update the average validation loss
        pred = model(data)
        loss = criterion(pred, target)
        valid_loss += ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
    if(epoch % 10 == 0):
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

    ## TODO: save the model if validation loss has decreased
    if valid_loss < valid_loss_min:
        if(epoch % 10 == 0):
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), "checkpoint.cpt")
        valid_loss_min = valid_loss

Epoch: 0 	Training Loss: 0.695210 	Validation Loss: 0.692871
Validation loss decreased (inf --> 0.692871).  Saving model ...
Epoch: 10 	Training Loss: 0.692205 	Validation Loss: 0.691015
Validation loss decreased (0.691256 --> 0.691015).  Saving model ...
Epoch: 20 	Training Loss: 0.686702 	Validation Loss: 0.683937
Validation loss decreased (0.685431 --> 0.683937).  Saving model ...
Epoch: 30 	Training Loss: 0.659876 	Validation Loss: 0.642457
Validation loss decreased (0.651474 --> 0.642457).  Saving model ...
Epoch: 40 	Training Loss: 0.548252 	Validation Loss: 0.487112
Validation loss decreased (0.503515 --> 0.487112).  Saving model ...
Epoch: 50 	Training Loss: 0.435167 	Validation Loss: 0.365942
Validation loss decreased (0.372897 --> 0.365942).  Saving model ...
Epoch: 60 	Training Loss: 0.349718 	Validation Loss: 0.288616
Validation loss decreased (0.295855 --> 0.288616).  Saving model ...
Epoch: 70 	Training Loss: 0.295995 	Validation Loss: 0.247370
Validation loss decreased (

In [31]:
# monitor test loss and accuracy
test_loss = 0.
correct = 0.
total = 0.

model.eval()
for batch_idx, (data, target) in enumerate(loaders['test']):
    # move to GPU
    if use_cuda:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update average test loss 
    test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
    # convert output probabilities to predicted class
    pred = output.data.max(1, keepdim=True)[1]
    # compare predictions to true label
    correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
    total += data.size(0)

print('Test Loss: {:.6f}\n'.format(test_loss))

print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
    100. * correct / total, correct, total))

Test Loss: 0.146854


Test Accuracy: 94% (20006/21159)
